# Distributed Arrays with JAX

In [ ]:
import jax
import numpy as np

In [ ]:
gpus = jax.devices('gpu')

In [ ]:
for i, d in enumerate(gpus):
    print(f"Device {i}: {d}")

In [ ]:
jax.local_device_count()

### `Sharding` describes how array values are laid out in memory across devices

In [ ]:
mesh = jax.sharding.Mesh(jax.devices(), axis_names=['x'])

In [ ]:
x = jax.random.normal(jax.random.key(0), (16000, 16000))

In [ ]:
y = jax.device_put(x, jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec('x', None)))

In [ ]:
y.sum(axis=0)

In [ ]:
jax.debug.visualize_array_sharding(y)

In [ ]:
@jax.jit
def my_complex_sin(z):
    y = jax.numpy.sin(z) + jax.numpy.cos(z) + z ** 2

    return y.sum(axis=0)

In [ ]:
%time z = my_complex_sin(x).block_until_ready()

In [ ]:
%time z2 = my_complex_sin(y).block_until_ready()

In [ ]:
z2.sharding

### JAX takes sharding into account when performing computations

#### <mark>Hands-on:</mark> Perform some standard array operations (e.g Matrix-Matrix Multiplication), compare the execution with non-sharded arrays and check the resulting array sharding.